In [21]:
import datetime
import socket

from ib_async import *

hostname = socket.gethostname()
util.logToFile(f"{datetime.datetime.now().strftime('%Y-%m-%d')}-{hostname}.log")
util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=188)


<IB connected to 127.0.0.1:4001 clientId=188>

In [22]:
from IPython.display import clear_output

import datetime
import time
import os


from tools import (
    play_beep,
    print_account_summary,
    print_order,
)

from IPython.display import display
from IPython.core.display import HTML

play_beep()

display(HTML("<style>.output_subarea { overflow: auto; }</style>"))


from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)

# ib.reqPositions()

# ib.reqAccountSummary()
# ib.reqAllOpenOrders()
# ib.reqAccountUpdates()


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


print(ticker)

# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

# ib.cancelMktDepth(contract)

# trades = get_all_openorders(ib)
# open_trade = [trade for trade in trades if trade.order.permId == 449873795]
# print_order(open_trade)

# future = [
#     pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
# ][0]


def print_openorders():
    print("Open Orders")
    orders = ib.openOrders()
    orders.sort(key=lambda order: order.lmtPrice)

    # Order(orderId=658, clientId=3124, permId=342738244, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=17692.5, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True)

    for order in orders:
        print(
            f"{order.permId}\t{order.action}\t{order.orderType}\t{order.totalQuantity}\t{order.lmtPrice}\t{order.tif}"
        )


def print_summary():
    print("NQ Order Book")
    if ticker is not None and ticker.domBids is not None and ticker.domAsks is not None:
        for i in range(min(len(ticker.domBids), len(ticker.domAsks))):
            bid_size = ticker.domBids[i].size
            bid_price = ticker.domBids[i].price
            ask_price = ticker.domAsks[i].price
            ask_size = ticker.domAsks[i].size
            print(f"{bid_size:>8} {bid_price:>10} | {ask_price:<10} {ask_size:<8}")

    print()
    fills = [t.fills for t in ib.trades() if t.fills != []]
    executions = [f[0].execution for f in fills]
    print(f"Day Executions: {len(executions)}")
    print()
    print_openorders()
    print()
    future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"]
    for f in future:
        print(
            f"{f.contract.symbol} {f.position} @ {f.avgCost/float(contract.multiplier)}"
        )
    print()
    print_account_summary(ib=ib)
    print()


print_summary()


In [19]:
# default reset values
open_trade = None

# open_trade = ib.trades()[0]
close_trade = None

print(open_trade)
print(close_trade)


None
None


In [20]:
## SELL TO OPEN SCALP 0.1
########################################

sell_scalp = {
    "strategy": "SELL TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10,
    "pause_seconds": 90,
}

buy_scalp = {
    "strategy": "BUY TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "bid",
    "open_ticks": -10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 10,
    "pause_seconds": 90,
}


strategy_details = buy_scalp

print(strategy_details["strategy"])

while True:
    clear_output(wait=True)

    # first order of the strategy
    if open_trade is None and close_trade is None:
        action = strategy_details["open_action"]
        qty = strategy_details["open_qty"]

        if strategy_details["open_ref"] == "bid":
            price_ref = ticker.domBids[0].price
        elif strategy_details["open_ref"] == "ask":
            price_ref = ticker.domAsks[0].price
        elif strategy_details["open_ref"] == "mid":
            price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
        elif strategy_details["open_ref"] == "last":
            raise Exception("Not implemented")

        lmtPrice = (
            price_ref
            + strategy_details["open_ticks"] * strategy_details["tick_increment"]
        )
        print(
            f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
        )
        print()

        if strategy_details["open_type"] == "LIMIT":
            open_order = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
        else:
            raise Exception("Not implemented")

        open_trade = ib.placeOrder(contract, open_order)
        open_order_ts = datetime.datetime.now()

    print("OPEN ORDER::")
    print_order(open_trade)
    print()

    if open_trade is not None:
        if open_trade.orderStatus.status == "Submitted" and close_trade is None:
            print(
                f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})\n"
            )

            if datetime.datetime.now() - open_order_ts > datetime.timedelta(
                seconds=strategy_details["pause_seconds"]
            ):
                print("Cancelling order due to timeout:")
                ib.cancelOrder(open_trade.order)
                print()

        if open_trade.orderStatus.status == "Filled" and close_trade is None:
            action = strategy_details["close_action"]
            qty = strategy_details["close_qty"]

            if strategy_details["close_ref"] == "open_price_fill":
                price_ref = open_trade.orderStatus.avgFillPrice
            if strategy_details["close_ref"] == "bid":
                price_ref = ticker.domBids[0].price
            elif strategy_details["close_ref"] == "ask":
                price_ref = ticker.domAsks[0].price
            elif strategy_details["close_ref"] == "mid":
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
            elif strategy_details["close_ref"] == "last":
                raise Exception("Not implemented")

            lmtPrice = (
                price_ref
                + strategy_details["close_ticks"] * strategy_details["tick_increment"]
            )

            if strategy_details["close_type"] == "LIMIT":
                close_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception("Not implemented")

            close_trade = ib.placeOrder(contract, close_order)
            play_beep(500, 500)

        elif (
            open_trade.orderStatus.status == "Inactive"
            or open_trade.orderStatus.status == "Cancelled"
        ) and close_trade is None:
            print("***** order is inactive *****")
            print(open_trade.log)
            print("*****************************")
            open_trade = None

    print(f"CLOSE ORDER::")
    print_order(close_trade)
    print()

    if close_trade is not None:
        if close_trade.orderStatus.status == "Filled":
            play_beep(2500, 500)
            print(
                "Close trade filled @ {}\n".format(close_trade.orderStatus.avgFillPrice)
            )
            open_trade = None
            close_trade = None
            ib.accountSummary()
            ib.sleep(strategy_details["pause_seconds"])

    print("ALL OPEN ORDERS::")
    print_openorders()
    print()

    if ticker is not None and ticker.domBids is not None and ticker.domAsks is not None:
        for i in range(min(len(ticker.domBids), len(ticker.domAsks))):
            bid_size = ticker.domBids[i].size
            bid_price = ticker.domBids[i].price
            ask_price = ticker.domAsks[i].price
            ask_size = ticker.domAsks[i].size
            print(f"{bid_size:>8} {bid_price:>10} | {ask_price:<10} {ask_size:<8}")

    print()
    fills = [t.fills for t in ib.trades() if t.fills != []]
    executions = [f[0].execution for f in fills]
    print(f"Day Executions: {len(executions)}")

    future = [pos for pos in ib.positions()]
    for f in future:
        print(
            f"{f.contract.symbol} {f.position} @ {f.avgCost/float(contract.multiplier)}"
        )
    print()
    print_account_summary(ib=ib)

    ib.sleep(1)


OPEN ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	0	PendingSubmit	BUY	0.0	0.0		18294.5	

CLOSE ORDER::
None

ALL OPEN ORDERS::
Open Orders
941324696	BUY	LMT	1.0	18086.0	GTC
623820843	BUY	LMT	1.0	18282.75	
0	BUY	LMT	1	18294.5	
0	BUY	LMT	1	18298.75	

    13.0    18295.0 | 18295.5    4.0     
     7.0   18294.75 | 18295.75   8.0     
    14.0    18294.5 | 18296.0    6.0     
    22.0   18294.25 | 18296.25   12.0    
    20.0    18294.0 | 18296.5    12.0    

Day Executions: 82
BA 254.0 @ 10.1605561025
EDV 650.0 @ 3.885403845
RSX 100.0 @ 0.34449885
HYG 50.0 @ 0.6223425
AAPL 4.0 @ 9.109842500000001
QQQ 1.0 @ 25.051845
NVDA 2.0 @ 17.471155
GOOGL 10.0 @ 2.5723425
PINS 40.0 @ 0.6723425
TSLA -1.0 @ 7.374460000000001
NQ -6.0 @ 18297.89352443
NVDA 1.0 @ 25.051845
QQQ 2.0 @ 10.384842500000001
AMZN 4.0 @ 5.7843425

AccountType : INDIVIDUAL
Cushion : 0.40596
LookAheadNextChange : 1715630400
AccruedCash : 0.00
AvailableFunds : 55201.15
BuyingPower : 256619.14
EquityWithLoanValue

KeyboardInterrupt: 

In [ ]:
fills = [t.fills for t in ib.trades() if t.fills != []]
executions = [f[0].execution for f in fills]
util.df(executions)


In [ ]:
ib.cancelOrder(open_trade.order)


In [ ]:
print_all_openorders(ib=ib)


In [ ]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != "NQ":
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )


In [ ]:
trades_df = util.df(ib.trades())
trades_df


In [ ]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect("trades.db")

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create the "trades" table if it doesn't exist
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS trades (
        contract TEXT,
        order TEXT,
        orderStatus TEXT,
        fills TEXT,
        log TEXT,
        advancedError TEXT
    )
"""
)

# Convert the DataFrame to a list of tuples
trades_data = trades_df.to_records(index=False).tolist()

# Insert the data into the "trades" table
cursor.executemany(
    """
    INSERT INTO trades (contract, order, orderStatus, fills, log, advancedError)
    VALUES (?, ?, ?, ?, ?, ?)
""",
    trades_data,
)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [ ]:
# get all the trades where fills is not []
fills = trades_df[trades_df.fills.map(len) > 0]

fills_list = list(fills["order"])

# loop through fills and convert each member of this data type to a row in a df
#  Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')
